In [ ]:
def installRoot():
  !pip install -q condacolab

  import condacolab

  condacolab.install()

  !sed -i '/cudatoolkit/d' /usr/local/conda-meta/pinned
  !conda config --add channels conda-forge
  !conda config --set channel_priority strict
  !conda install root_base
  #!conda install pytorch torchvision -c pytorch -c conda-forge


In [ ]:
try:
  import ROOT
except:
  print("ROOT not installed. Installing ROOT. The notebook will be restarted once the installation succeeds.")
  installRoot()

#the notebook may crash after the first run of this cell.
#When you run the cell again you should see “Welcome to JupyROOT 6.30/04”.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import ROOT
import os

# Step 2: Specify the path in your Google Drive to save the data
# By default, a 'Colab Notebooks' folder is created in your Drive. You can create a subfolder if needed.
# Define the path where you want to save the data

drive_path = '/content/drive/My Drive/Alexis_exercises/'

# Create the folder if it doesn't exist
os.makedirs(drive_path, exist_ok=True)

# Define the directory path for saving plots in Google Drive
plot_dir = "/content/drive/My Drive/Colab Notebooks/Plots/"

os.makedirs(plot_dir, exist_ok=True)


In [ ]:
from ROOT import RooRealVar, RooBreitWigner, RooNumConvPdf, RooGaussian, RooExponential, RooDataSet, RooAddPdf, RooArgList, RooRandom, RooMinimizer, RooPlot, TCanvas, kGreen, kRed, kBlue, kDashed
from ROOT import RooArgSet, RooFitResult, RooFit
import time

name = "10000"
events = int(name)
bins = 200
seed = int(time.time())

print("Seed is ", seed)
xvar = RooRealVar("xvar", "", -10, 10)
xvar.setBins(bins)

# Breit Wigner Signal
mean = RooRealVar("m", "mean", 0.2, -1, 1)
gamma = RooRealVar("#Gamma", "gamma", 2, 0.1, 5)
signal = RooBreitWigner("BW", "BW signal", xvar, mean, gamma)

# Gaussian Resolution Function
zero = RooRealVar("zero", "Gaussian resolution mean", 0.)
sigma = RooRealVar("#sigma", "sigma", 1.5, 0.1, 5)
resol = RooGaussian("resol", "Gaussian resolution", xvar, zero, sigma)

# Background
alpha = RooRealVar("#alpha", "Exponential Parameter", -0.05, -2.0, 0.0)
bkg = RooExponential("Bkg", "Bkg", xvar, alpha)

# Gaussian + BW convolution
convolution = RooNumConvPdf("convolution", "BW (X) gauss", xvar, signal, resol)

# TotalPdf = Gaussian + Bkg
sigfrac = RooRealVar("sig1frac", "fraction of component 1 in signal", 0.5, 0., 1.)
total = RooAddPdf("totalPDF", "totalPDF", RooArgList(convolution, bkg), sigfrac)

print(f"\n------Generating {name} events\n")
print("\n------Remember: initial values for fitting step are the generated (true) values in generation ---")


Seed is  1744295813

------Generating 10000 events


------Remember: initial values for fitting step are the generated (true) values in generation ---


In [ ]:

RooRandom.randomGenerator().SetSeed(seed)
data = total.generate(xvar, events)

data.write(f"{plot_dir}_{events}_events.txt")

print(f"\nFitting {name} events\n")



Fitting 10000 events

[#1] INFO:NumericIntegration -- RooRealIntegral::init(convolution_Int[xvar]) using numeric integrator RooIntegrator1D to calculate Int(xvar)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(convolution_Int[xvar]) using numeric integrator RooIntegrator1D to calculate Int(xvar)
[#1] INFO:DataHandling -- RooDataSet::write(totalPDFData) writing ASCII file /content/drive/My Drive/Colab Notebooks/Plots/_10000_events.txt


[#1] INFO:NumericIntegration -- RooRealIntegral::init(convolution_Int[xvar]) using numeric integrator RooIntegrator1D to calculate Int(xvar)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(convolution_Int[xvar]) using numeric integrator RooIntegrator1D to calculate Int(xvar)
[#1] INFO:DataHandling -- RooDataSet::write(totalPDFData) writing ASCII file /content/drive/My Drive/Colab Notebooks/Plots/_10000_events.txt


In [ ]:

start_time = time.time()
start_cpu = time.process_time()

nll = total.createNLL(data)
minimizer = RooMinimizer(nll)
minimizer.setMinimizerType('Minuit2')
minimizer.migrad()

end_time = time.time()
end_cpu = time.process_time()

print("\n--------------------minimization done; now recalculating the uncertainties----------")

minimizer.hesse()
print("\n--------------------fit done; check best estimates for the model parameters---------")



--------------------minimization done; now recalculating the uncertainties----------

--------------------fit done; check best estimates for the model parameters---------
[#1] INFO:Fitting -- RooAbsPdf::fitTo(totalPDF) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using CPU computation library compiled with -mavx2
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_totalPDF_totalPDFData) Summation contains a RooNLLVar, using its error level
Minuit2Minimizer: Minimize with max-calls 2500 convergence for edm < 1 strategy 1
[#1] INFO:NumericIntegration -- RooRealIntegral::init(convolution_Int[xvar]) using numeric integrator RooIntegrator1D to calculate Int(xvar)
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 28418.7371097177529
Edm   = 0.000347202659914105742
Nfcn  = 145
#Gamma	  = 2.15903	 +/-  0.302485	(limited)
#alpha	  = -0.0506747	 +/-  0.00359068	(limited)
#sigma	  = 1.48465	 +/-  0.104401	(limited)
m	  = 0.278459	 +/-

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       28420.64975 Edm =       2.207806685 NCalls =     21
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 28420.64975
  Edm           : 2.207806685
  Internal parameters:	[    -0.2264194372      1.253235898    -0.4429110441     0.2013579208                0]	
  Internal gradient  :	[     -1.401658934     -88.61442424     -12.88065305     -37.77138259     -36.08932189]	
  Internal covariance matrix:
[[   0.0044991177              0              0              0              0]
 [              0  0.00015700341              0              0              0]
 [              0              0   0.0011287748              0              0]
 [              0              0              0   0.0045763135              0]
 [              0              0              0              0  0.00067052394]]]
Info in <Minuit2>: VariableMetricBuilde

In [ ]:
result = minimizer.save()

print("Fit results:")
for param in result.floatParsFinal():
    print(f"{param.GetName()} = {param.getVal():.3f} +/- {param.getError():.3f}")



Fit results:
#Gamma = 2.159 +/- 0.492
#alpha = -0.051 +/- 0.004
#sigma = 1.485 +/- 0.164
m = 0.278 +/- 0.049
sig1frac = 0.514 +/- 0.023


In [ ]:

myC = TCanvas("RooCanvas", "Roofit Canvas", 1000, 750)

frame = xvar.frame()
frame.SetTitle(f"RooFit : {events} events")
frame.SetYTitle("# of events")

data.plotOn(frame)
total.plotOn(frame, RooFit.LineColor(ROOT.kGreen))
total.plotOn(frame, RooFit.Components(RooArgSet(convolution)), RooFit.LineColor(ROOT.kRed))
total.plotOn(frame, RooFit.Components(RooArgSet(bkg)), RooFit.LineColor(ROOT.kBlue), RooFit.LineStyle(kDashed))
total.paramOn(frame, RooFit.Layout(0.75, 0.99, 0.99))
frame.getAttText().SetTextSize(0.028)

frame.Draw()
myC.SaveAs(f"plots/RooConvGen_{name}.png")

#print("\n------------------------------")
#print(f"Total CPU time: {end_time - start_time:.3f} seconds")
#print("------------------------------\n")

print("\n------------------------------")
print(f"Total time: {end_time - start_time:.3f} seconds")

# WRONG ->   print(f"Total CPU time: {(end_cpu - start_cpu) / os.sysconf(os.sysconf_names['SC_CLK_TCK']):.3f} seconds")
print(f"Total CPU time: {(end_cpu - start_cpu) :.3f} seconds")
print("------------------------------\n")

#### If Total Time and CPU Time are close it means we are exclusive user of the Google CPU !


------------------------------
Total time: 101.735 seconds
Total CPU time: 1.005 seconds
Total CPU time: 100.534 seconds
------------------------------

[#1] INFO:NumericIntegration -- RooRealIntegral::init(convolution_Int[xvar]) using numeric integrator RooIntegrator1D to calculate Int(xvar)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(totalPDF) directly selected PDF components: (convolution)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(totalPDF) indirectly selected PDF components: (BW,resol)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(convolution_Int[xvar]) using numeric integrator RooIntegrator1D to calculate Int(xvar)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(totalPDF) directly selected PDF components: (Bkg)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(totalPDF) indirectly selected PDF components: ()
[#1] INFO:NumericIntegration -- RooRealIntegral::init(convolution_Int[xvar]) using numeric integrator RooIntegrator1D to calculate Int(xvar)


python ERROR: cannot open image file "plots/RooConvGen_10000.png" for writing. Please check permissions.
Error in <TASImage::WriteImage>: error writing file plots/RooConvGen_10000.png


[#1] INFO:NumericIntegration -- RooRealIntegral::init(convolution_Int[xvar]) using numeric integrator RooIntegrator1D to calculate Int(xvar)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(totalPDF) directly selected PDF components: (convolution)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(totalPDF) indirectly selected PDF components: (BW,resol)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(convolution_Int[xvar]) using numeric integrator RooIntegrator1D to calculate Int(xvar)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(totalPDF) directly selected PDF components: (Bkg)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(totalPDF) indirectly selected PDF components: ()
[#1] INFO:NumericIntegration -- RooRealIntegral::init(convolution_Int[xvar]) using numeric integrator RooIntegrator1D to calculate Int(xvar)


python ERROR: cannot open image file "plots/RooConvGen_10000.png" for writing. Please check permissions.
Error in <TASImage::WriteImage>: error writing file plots/RooConvGen_10000.png
